# Data preprocessing

The code in this notebook takes the input pictures and applies the mediapipe hands solution to find the relevant coordinates and saves them as a .csv file which can be used by the AI model to train.

## Imports

In [13]:
!pip install mediapipe

import pandas as pd
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np

## Download database

In [5]:
# kaggle API
!mkdir ~/.kaggle
!echo '{"username":"spacewaiker","key":"062bca10262b87f5b8498c589cdc65c6"}' > ~/.kaggle/kaggle.json

In [6]:
# download dataset:
!kaggle datasets download grassknoted/asl-alphabet
# extract files (-q disables writing status message for every file)
!unzip -q asl-alphabet.zip
!ls

 99% 1.02G/1.03G [00:32<00:00, 68.6MB/s]
100% 1.03G/1.03G [00:32<00:00, 34.0MB/s]
asl_alphabet_test  asl_alphabet_train  asl-alphabet.zip  drive	sample_data


Symbols:

|Command  |ASL letters|Sign           |
|---------|-----------|---------------|
|Line     |L          |"L" shape      |
|Ellipse  |A, S       |Fist           |
|Rectangle|G          |Point right    |
|Triangle |K, V       |Two fingers up |
|Move     |B          |Palm           |
|Delete   |R          |Crossed fingers|

## Define functions to use later

In [7]:
'''
Function to return the file names with the following format:
A/A1234.jpg     if letter = 'A'  and   i = 1234
'''
def get_file_names(letter, begin, end):
    files = []
    for i in range(begin, end+1):
        files.append(f'{letter}/{letter}{i}.jpg')
    
    return files

'''
Function that applies the mediapipe hands solution to an image.
img is the image file name and hands is the hands instance to use
'''
def apply_hands(row):
    global hands
    img = row.name
    result = row.copy()

    image = cv2.imread(DIR + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    landmk = hands.process(image).multi_hand_world_landmarks

    if landmk:    # success
        landmarks = landmk[0]
        list_tuples = [(i.x, i.y, i.z) for i in landmarks.landmark]
        result[1:64] = [i for t in list_tuples for i in t]

    return result

In [8]:
!pip show mediapipe

Name: mediapipe
Version: 0.8.9
Summary: MediaPipe is the simplest way for researchers and developers to build world-class ML solutions and applications for mobile, edge, cloud and the web.
Home-page: https://github.com/google/mediapipe
Author: The MediaPipe Authors
Author-email: mediapipe@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: matplotlib, opencv-contrib-python, numpy, attrs, wheel, protobuf, six, absl-py
Required-by: 


In [16]:
# Mediapipe stuff
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# get file names
file_names = pd.DataFrame()

file_names = file_names.append([['Line',      file] for file in get_file_names('L', 1, 3000)])
file_names = file_names.append([['Ellipse',   file] for file in get_file_names('A', 1, 1500) + get_file_names('S', 1, 1500)])
file_names = file_names.append([['Rectangle', file] for file in get_file_names('G', 1, 3000)])
file_names = file_names.append([['Triangle',  file] for file in get_file_names('K', 1, 1500) + get_file_names('V', 1, 1500)])
file_names = file_names.append([['Move',      file] for file in get_file_names('B', 1, 3000)])
file_names = file_names.append([['Delete',    file] for file in get_file_names('R', 1, 3000)])

file_names.rename(columns={0: 'Command', 1: 'File'}, inplace=True)
file_names.set_index('File', inplace=True)
print(file_names)

            Command
File               
L/L1.jpg       Line
L/L2.jpg       Line
L/L3.jpg       Line
L/L4.jpg       Line
L/L5.jpg       Line
...             ...
R/R2996.jpg  Delete
R/R2997.jpg  Delete
R/R2998.jpg  Delete
R/R2999.jpg  Delete
R/R3000.jpg  Delete

[18000 rows x 1 columns]


In [17]:
from sklearn.model_selection import train_test_split

DIR = 'asl_alphabet_train/asl_alphabet_train/'

cols = ['Command'] + [i for i in range(21*3)]
train_x = pd.DataFrame(np.zeros((16_200, 64)), columns=cols)
test_x = pd.DataFrame(np.zeros((1800, 64)), columns=cols)


train_x.index, test_x.index, train_x['Command'], test_x['Command'] = train_test_split(
    file_names.index, file_names['Command'], test_size=0.1, random_state=1)


# Loop over every file:
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.3
) as hands:
    train_x = train_x.apply(apply_hands, axis=1)

print(train_x)

with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.3
) as hands:
    test_x = test_x.apply(apply_hands, axis=1)

print(test_x)

Index(['L/L1.jpg', 'L/L2.jpg', 'L/L3.jpg', 'L/L4.jpg', 'L/L5.jpg', 'L/L6.jpg',
       'L/L7.jpg', 'L/L8.jpg', 'L/L9.jpg', 'L/L10.jpg',
       ...
       'R/R2991.jpg', 'R/R2992.jpg', 'R/R2993.jpg', 'R/R2994.jpg',
       'R/R2995.jpg', 'R/R2996.jpg', 'R/R2997.jpg', 'R/R2998.jpg',
       'R/R2999.jpg', 'R/R3000.jpg'],
      dtype='object', name='File', length=18000)
File
L/L1.jpg         Line
L/L2.jpg         Line
L/L3.jpg         Line
L/L4.jpg         Line
L/L5.jpg         Line
                ...  
R/R2996.jpg    Delete
R/R2997.jpg    Delete
R/R2998.jpg    Delete
R/R2999.jpg    Delete
R/R3000.jpg    Delete
Name: Command, Length: 18000, dtype: object
               Command         0         1  ...        60        61        62
File                                        ...                              
S/S1455.jpg    Ellipse -0.013543  0.081017  ... -0.024496  0.036506 -0.024996
V/V38.jpg     Triangle  0.000000  0.000000  ...  0.000000  0.000000  0.000000
A/A926.jpg     Ellipse  0.0030

In [18]:
# sort dfs
train_x.sort_index(inplace=True)
test_x.sort_index(inplace=True)

print(train_x)
print(test_x)

              Command         0         1  ...        60        61        62
File                                       ...                              
A/A1.jpg      Ellipse -0.003756  0.080883  ... -0.012595  0.045449 -0.023966
A/A10.jpg     Ellipse -0.005596  0.080807  ... -0.010209  0.049343 -0.019856
A/A100.jpg    Ellipse  0.000000  0.000000  ...  0.000000  0.000000  0.000000
A/A1000.jpg   Ellipse -0.002892  0.081772  ... -0.026531  0.037126 -0.027262
A/A1001.jpg   Ellipse  0.005875  0.083854  ... -0.011564  0.046879 -0.030367
...               ...       ...       ...  ...       ...       ...       ...
V/V989.jpg   Triangle  0.007171  0.090853  ... -0.012653  0.039348 -0.019005
V/V99.jpg    Triangle  0.000453  0.084601  ... -0.013265  0.031576 -0.022190
V/V990.jpg   Triangle  0.008918  0.090453  ... -0.012972  0.038318 -0.018570
V/V996.jpg   Triangle  0.010703  0.088921  ... -0.012597  0.037550 -0.019387
V/V998.jpg   Triangle  0.008637  0.089558  ... -0.013161  0.035479 -0.019120

In [19]:
# save dfs
train_x.to_csv("drive/MyDrive/MAISFinalProject/train_x.csv")
test_x.to_csv('drive/MyDrive/MAISFinalProject/test_x.csv')